In [104]:
import pickle
import os
import numpy as np
import os
import cv2
import logging
import uuid
import sys
import heapq
import numpy as np
import sys, faiss



def GET_FAISS_RESOURCES():
    return None

def GET_FAISS_INDEX(index_path):
    import faiss
    index = faiss.read_index(index_path)
    return index

def GET_FAISS_ID_TO_VECTOR(ids_vectors_path):
    if not os.path.exists(ids_vectors_path):
      return None
    with open(ids_vectors_path, 'rb') as f:
      index_dict = pickle.load(f)
    def id_to_vector(id_):
      try:
        return index_dict[id_]
      except:
        pass
    return id_to_vector


NOR_X = 512
NOR_Y = 384

PHASH_X = 8
PHASH_Y = 8

SIFT_DIMENSIONS = 128

NUM_FEATURES = 100
isAddPhash = False

bow_num_words = 1000
dictionary_path = '/faiss-web-service/resources/dictionary'

INDEX_KEY = "IDMap,PCA128,IVF2048,PQ16"

TOP_N = 5
SIMILARITY = 5

In [105]:
def calc_phash(gray_image):
  img = gray_image


  img = cv2.resize(img, (PHASH_X, PHASH_Y), interpolation=cv2.INTER_CUBIC)
  h, w = img.shape[:2]
  vis0 = np.zeros((h, w), np.float32)
  vis0[:h, :w] = img
  vis1 = cv2.dct(cv2.dct(vis0))
  vis1.resize(PHASH_X, PHASH_Y)
  img_list = vis1.flatten()
  avg = sum(img_list) * 1. / len(img_list)
  avg_list = [np.float32(0) if i < avg else np.float32(1) for i in img_list]
  return np.matrix(avg_list).flatten()

In [106]:
def adddPhash(gray_image, des):
  phash = calc_phash(gray_image)
  n, d = des.shape
  phash_mat = phash
  for i in range(n - 1):
    phash_mat = np.vstack((phash_mat, phash))
  des = np.hstack((des, phash_mat))
  return des

In [107]:
def calc_sift(sift, image_file):
  if not os.path.isfile(image_file):
    logging.error('Image:{} does not exist'.format(image_file))
    return -1, None
  try:
    image_o = cv2.imread(image_file)
  except:
    logging.error('Open Image:{} failed'.format(image_file))
    return -1, None
  if image_o is None:
    logging.error('Open Image:{} failed'.format(image_file))
    return -1, None
  image = cv2.resize(image_o, (NOR_X, NOR_Y))
  if image.ndim == 2:
    gray_image = image
  else:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray_image, (5, 5), 0)
    equalized = cv2.equalizeHist(blurred)
      
  kp, des = sift.detectAndCompute(equalized, None)
  if isAddPhash:
    des = adddPhash(gray_image, des)
  sift_feature = np.matrix(des)
  return 0, sift_feature

In [108]:
def get_sift():
  return cv2.SIFT_create()

#nfeatures=NUM_FEATURES, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6

In [109]:
def get_vectors(sift, image):
  return calc_sift(sift, image)

In [110]:
def iterate_files(_dir):
  result = []
  for root, dirs, files in os.walk(_dir, topdown=True):
    for fl in files:
      if fl.endswith("jpg") or fl.endswith(".JPG"):
        result.append(os.path.join(root, fl))
  '''for files in os.walk(_dir, topdown=True):
    (x, y, z) = files
  for fl in z:
    if fl.endswith("jpg") or fl.endswith("JPG"):
      result.append(os.path.join(x, fl))'''
  return result

In [111]:
img_list = iterate_files(r"D:\API_APP\FAST_FAISS_HNSW\processed")

In [112]:
files = os.walk(r"D:\API_APP\FAST_FAISS_HNSW\processed", topdown=True)

In [113]:
for f in files:
  (x, y, z) = f

In [114]:
print(len(z))

1929


In [115]:
res1 = []
for fl in z:
  if fl.endswith("jpg") or fl.endswith("JPG"):
    res1.append(os.path.join(x, fl))

In [116]:
print(len(res1))

1929


In [118]:
dimensions = SIFT_DIMENSIONS
if isAddPhash:
  dimensions += PHASH_X * PHASH_Y
index = faiss.index_factory(dimensions, INDEX_KEY)

images_list = iterate_files(r"D:\API_APP\FAST_FAISS_HNSW\processed")
ids_count = 0
index_dict = {}
ids = None
features = np.matrix([])
sift = get_sift()

In [119]:
for file_name in images_list:
  ret, sift_feature = calc_sift(sift, file_name)
  if ret == 0 and sift_feature.any():
    image_dict = {ids_count: (file_name, sift_feature)}
    index_dict.update(image_dict)
    #print(sift_feature.shape[0])
    ids_list = np.linspace(ids_count, ids_count, num=sift_feature.shape[0], dtype="int64")
    ids_count += 1
    if features.any():
      features = np.vstack((features, sift_feature))
      ids = np.hstack((ids, ids_list))
    else:
      features = sift_feature
      ids = ids_list
    if ids_count % 9000000 == 8999999:
      if not index.is_trained and INDEX_KEY != "IDMap,Flat":
        index.train(features)
      index.add_with_ids(features, ids)
      ids = None
      features = np.matrix([])
if features.any():
  if not index.is_trained and INDEX_KEY != "IDMap,Flat":
    index.train(features)
    print('done training')
  index.add_with_ids(features, ids)
  print('done adding')

done training
done adding


In [120]:
index_dict[50]

('D:\\API_APP\\FAST_FAISS_HNSW\\processed\\IMG_0068.JPG',
 matrix([[  0.,   0.,   0., ...,   0.,   0.,   1.],
         [  8.,  21., 118., ...,   0.,   0.,   0.],
         [  1.,   0.,   0., ...,   0.,   0.,   0.],
         ...,
         [  0.,   1.,  20., ...,   0.,   0.,   0.],
         [  0.,  15., 128., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,  16.,   5.,   2.]], dtype=float32))

In [121]:
ids_vectors_path = 'ids_path_vectors'

In [122]:
print(index.is_trained)
print(index.ntotal)

True
3844010


In [123]:
with open(ids_vectors_path, 'wb+') as f:
  try:
    pickle.dump(index_dict, f, True)
  except EnvironmentError as e:
    logging.error('Failed to save index file error:[{}]'.format(e))
    f.close()
  except RuntimeError:
    logging.error('Failed to save index file error:[{}]'.format(v))
f.close()

In [124]:
id_to_vector = GET_FAISS_ID_TO_VECTOR('ids_path_vectors')

In [125]:
class FaissIndex(object):
  def __init__(self, index, id_to_vector):
    assert index
    self.index = index
    self.id_to_vector = id_to_vector
    self.sift = get_sift()
  
  def search_by_ids(self, ids, k):
    vectors = [self.id_to_vector(id_)[1] for id_ in ids]
    results = self.__search__(ids, vectors, k + 1)
    return results

  def search_by_vectors(self, vectors, k):
    vectors = read_array(vectors, SIFT_DIMENSIONS)
    count = vectors.shape[0]
    vectors = np.vstack((vectors, vectors))
    vectors = vectors[0:count, :]
    print(vectors.shape)
    ids = [None]
    results = self.__search__(ids, [vectors], k)
    return results

  def search_by_image(self, image, k):
    ids = [None]
    ret, vectors = get_vectors(self.sift, image)
    print(ret)
    results = self.__search__(ids, [vectors], k)
    return results

  def __search__(self, ids, vectors, topN):
    def neighbor_dict_with_path(id_, file_path, score):
      return {'id': float(id_), 'file_path': file_path, 'score': score}
    def neighbor_dict(id_, score):
      return {'id': float(id_), 'score': score}
    def result_dict_str(id_, neighbors):
      return {'id': id_, 'neighbors': neighbors}
    results = []
    need_hit = SIMILARITY
    for id_, siftfeature in zip(ids, vectors):
      scores, neighbors = self.index.search(siftfeature, k=topN) if siftfeature.size > 0 else ([], [])
      print(neighbors)
      n, d = neighbors.shape
      result_dict = {}
      for i in range(n):
        l = np.unique(neighbors[i]).tolist()
        for r_id in l:
          if r_id != -1:
            score = result_dict.get(r_id, 0)
            score += 1
            result_dict[r_id] = score
      h = []
      for k in result_dict:
        v = result_dict[k]
        if v >= need_hit:
          if len(h) < topN:
            heapq.heappush(h, (v, k))
          else:
            heapq.heappushpop(h, (v, k))
      result_list = heapq.nlargest(topN, h, key=lambda x: x[0])
      neighbors_scores = []
      for e in result_list:
        confidence = e[0] * 100 / n
        if self.id_to_vector:
          print("A")
          file_path = self.id_to_vector(e[1])[0]
          neighbors_scores.append(neighbor_dict_with_path(e[1], file_path, str(confidence)))
        else:
          neighbors_scores.append(neighbor_dict(e[1], str(confidence)))
      results.append(result_dict_str(id_, neighbors_scores))
    return results

In [126]:
fs2 = FaissIndex(index=index, id_to_vector=id_to_vector)

In [179]:
fs2.search_by_image("./Not_DB/7.jpg", 3)

0
[[1616  701  598]
 [1828   16 1574]
 [ 525  791  319]
 ...
 [ 513  120   39]
 [ 424 1496 1897]
 [1218  829  830]]
A
A
A


[{'id': None,
  'neighbors': [{'id': 665.0,
    'file_path': 'D:\\API_APP\\FAST_FAISS_HNSW\\processed\\IMG_0850.JPG',
    'score': '0.6251860672819292'},
   {'id': 319.0,
    'file_path': 'D:\\API_APP\\FAST_FAISS_HNSW\\processed\\IMG_0407.JPG',
    'score': '0.5656445370646026'},
   {'id': 315.0,
    'file_path': 'D:\\API_APP\\FAST_FAISS_HNSW\\processed\\IMG_0403.JPG',
    'score': '0.5358737719559392'}]}]